Exercise 5 – Introspection
Objective
To see a practical use of introspection, and use a subset of the tools available.
Overview
This question is in two parts, with an optional third.  We are going to implement a feature that exists in several languages other than Python (e.g. Perl and Ruby) called interpolation – the ability to embed a variable name inside a quoted string and have it replaced by the variable's value. It is a templating system by another name.
Questions
1.	Ruby supports embedded code inside a text string, for example:

x = 42
y = 37
print "x: #{x} y: #{y}"
Gives:
x: 42 y: 37

Oh, wait, this is a Python course!  We can't do that in Python.  Or can we?
Your task is to implement a limited interpolation feature, using a named function.  It will not be feature rich – we don't have all day!   What this should give you is an idea of how introspection can be useful.
Take a look at inter.py.  The first tricky part of parsing a text string is extracting the tokens of interest.  Since you already know and love regular expressions(!?) we have spared you the effort.  By the way, a full RE for this task is considerably more complex, we are sticking to some basic functionality.
So, the user calls trans() to translate the text.  That in turn calls get_value(), passing the name of the object to be replaced (obj), and returning a string representation of that object.  The second parameter, *trans, is used in the second part of this question.  Implement the get_value() function.
Hints:
a)	sys._getframe() enables you to get a frame object.  What should it's parameter be to get the caller's frame?
b)	The frame contains two dictionaries, f_locals and f_globals
c)	Don't forget to return a string.
When you test your code, note that some of the tests are for the next question.

2.	You will note that some of the tests did not work.  If the user specifies an attribute to the object then we extract the text after the "." and place that in the attr parameter.  So, for example:
"sys.platform"
obj will be "sys"
attr will be "platform"
Once extracted we are going to take the coward's way out and use eval() on the text.  You will have to reconstruct the string.

	If time allows…
3.	We will often want to use this trick in a print, for debugging for example.  So we are going to monkey-patch the print function.  Whether this is advisable in general code is debatable!
Python 2: 
Notice that we did :
from __future__ import print_function
It is much easier to monkey-patch the print() function than to hook into the print statement.*

Python 2 and Python 3:
Provide your own print function which calls trans().

Hints:
a)	A dummy function header is provided, commented out
b)	Note that the first parameter to print is a tuple
c)	To call the "real" print() function :
Python 2: use  __builtin__.print()
Python 3: use builtins.print()
Make sure that you use this for any tracing – otherwise you might get recursion!

* It can be done, but you hook into stdout.  Create your own class with a write() function.  This then calls sys.stdout.write() to write to the stream.

Solutions
1.	The Python 2 and Python 3 versions are almost the same, but in Python 2 the internal print is __builtin__.print():
def get_value(obj, *attr):
    gvars = sys._getframe(1).f_globals
    lvars = sys._getframe(1).f_locals
    retn = ""
    
    if obj in lvars:
        retn = lvars[obj]
    elif obj in gvars:
        retn = gvars[obj]
    else:
        builtins.print(obj, "not found")  # Python 3
    
    return str(retn)


2.	The Python 2 and Python 3 versions are almost the same, but in Python 2 the internal print is __builtin__.print():
def get_value(obj, *attr):
    gvars = sys._getframe(1).f_globals
    lvars = sys._getframe(1).f_locals
    retn = ""
    
    if obj in lvars:
        retn = lvars[obj]
        if attr:
            cmd  = obj + "." + attr[0]
            retn = eval(cmd, gvars, lvars)
            
    elif obj in gvars:
        retn = gvars[obj]
        if attr:
           cmd  = obj + "." + attr[0]
           retn = eval(cmd, gvars, lvars)
    else:
        builtins.print(obj, "not found")    # Python 3
    
    return str(retn)

 
3.	This time the print functions differ.
Python 2:
def print(*line, **kwargs):

    args = dict(sep=', ', end='\n', file=None)
    args.update(kwargs) 
    
    sep   = args['sep']
    end   = args['end']
    ofile = args['file']
    
    newlist = []
    for item in line:
        newlist.append(trans(str(item)))

    if ofile is None:
        ofile = sys.stdout

    __builtin__.print(sep.join(newlist),end=end,file=ofile)

Python 3:
def print(*line, sep=', ', end='\n', file=None, flush=False):

    newlist = []

    for item in line:
        newlist.append(trans(str(item)))

    if file is None:
        file = sys.stdout

    builtins.print(sep.join(newlist), end=end, file=file,
                   flush=flush)
